$$\frac{\partial{\vec{u}}}{\partial{t}} + (\vec{u} \cdot \vec{\nabla}{\vec{u}})\vec{u} = -\frac{1}{\rho}\vec{\nabla}{p} + \nu\vec{\nabla}^2\vec{u} + \vec{g}$$

$p$: pressure
$u$: velocity
$\rho$: the fluids density
$\nu$: the kinematic viscosity given by: $\frac{\eta}{\rho}$, with $\eta$ being the viscosity
$g$: gravity experienced by the fluid

The fluid will be incompressible given by:
$$ \vec{\nabla} \cdot \vec{u} = 0 $$
This means the fluid density should remain constant

Similar to that of the example at https://www.youtube.com/watch?v=BQLvNLgMTQE, here we will solve the Navier-Stokes equation in 2D using Chorin's Projection method.

First, by solving the momentum part without the pressure gradient
Second, by solving the pressure poisson's equation
Third, by correcting the velocities using the solution to the poisson's equation

